In [ ]:
import sys
from pathlib import Path
import importlib
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

PROJECT_ROOT = Path.cwd().resolve().parent if Path.cwd().name == 'analyses' else Path.cwd().resolve()
ANALYSES_DIR = PROJECT_ROOT / 'analyses'
OUT_DIR = ANALYSES_DIR / 'results'
OUT_DIR.mkdir(parents=True, exist_ok=True)

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
if str(ANALYSES_DIR) not in sys.path:
    sys.path.insert(0, str(ANALYSES_DIR))

import run_dual_loss_with_rl as runner
runner = importlib.reload(runner)

print('Project root:', PROJECT_ROOT)
print('Output dir:', OUT_DIR)
print('Runner:', runner.__file__)


## Configuration


In [ ]:
SCENARIO = 'discriminating'
HORIZONS = [1, 4, 8]
T = 1400
WINDOW = 150

TUNING_SEEDS = [0, 1, 2, 3]
TEST_SEEDS = [4, 5, 6, 7, 8, 9]
N_TRIALS = 35

LINEX_A = 1.0
KAPPA_GRID = np.array([0.01, 0.03, 0.1, 0.3, 1.0, 3.0, 8.0], dtype=float)

LOSS_SECTIONS = [('mse', 'squared', 'mse'), ('linex', 'linex', 'linex')]
METHODS = list(runner.ot.DEFAULT_METHOD_PARAMS.keys())

print('Methods:', METHODS)


## Tune Regular Methods By Horizon (Both Loss Sections)


In [ ]:
tuning_rows = []
tuned_by_loss_h = {}

for loss_section, ens_loss, objective_metric in LOSS_SECTIONS:
    tuned_by_h = {}
    print(f'\n=== Tuning section: {loss_section.upper()} ===')
    for h in HORIZONS:
        slices_h = []
        for seed in TUNING_SEEDS:
            data_t, forecasts_t, _, s_unc_t = runner.simulator.make_environment_and_forecasts(
                T=T, horizons=HORIZONS, window=WINDOW, seed=seed, include_oracle=False, scenario=SCENARIO
            )
            y_t, F_t, s_t = runner.align_for_horizon(data_t['pi'], forecasts_t[h], s_unc_t, h)
            slices_h.append((y_t, F_t, s_t))

        tuned_h = runner.ot.tune_all_methods_optuna(
            data_slices=slices_h,
            methods=METHODS,
            n_trials=N_TRIALS,
            seed=42 + 100 * h,
            loss=ens_loss,
            linex_a=LINEX_A,
            objective_metric=objective_metric,
        )

        tuned_by_h[h] = {m: dict(tuned_h[m].best_params) for m in METHODS}
        for m in METHODS:
            tuning_rows.append({
                'loss_section': loss_section,
                'horizon': float(h),
                'method': m,
                'best_params': str(tuned_h[m].best_params),
                'tuning_objective': float(tuned_h[m].best_value),
            })

    tuned_by_loss_h[loss_section] = tuned_by_h

tuning_df = pd.DataFrame(tuning_rows).sort_values(['loss_section', 'horizon', 'tuning_objective']).reset_index(drop=True)
display(tuning_df)


## Evaluate On Test Seeds (Regular + RL Methods)


In [ ]:
all_rows = []
all_diag_rows = []

for loss_section, _, _ in LOSS_SECTIONS:
    print(f'\n=== Evaluating section: {loss_section.upper()} ===')
    for seed in TEST_SEEDS:
        rows, diag_rows = runner.evaluate_one_seed(
            seed=seed,
            horizons=HORIZONS,
            T=T,
            window=WINDOW,
            scenario=SCENARIO,
            loss_section=loss_section,
            linex_a=LINEX_A,
            params_map_by_h=tuned_by_loss_h[loss_section],
            kappa_grid=KAPPA_GRID,
        )
        all_rows.extend(rows)
        all_diag_rows.extend(diag_rows)

summary_rows = runner.aggregate(all_rows)

detailed_df = pd.DataFrame(all_rows).sort_values(['loss_section', 'horizon', 'seed', 'method']).reset_index(drop=True)
summary_df = pd.DataFrame(summary_rows).sort_values(['loss_section', 'horizon', 'objective_mean']).reset_index(drop=True)
diag_df = pd.DataFrame(all_diag_rows).sort_values(['loss_section', 'horizon', 'seed', 'method', 't']).reset_index(drop=True)

print('Detailed rows:', len(detailed_df))
print('Summary rows:', len(summary_df))
print('Diagnostics rows:', len(diag_df))
display(summary_df)


## Save Outputs


In [ ]:
detailed_csv = OUT_DIR / 'dual_loss_full_detailed.csv'
summary_csv = OUT_DIR / 'dual_loss_full_summary.csv'
tuning_csv = OUT_DIR / 'dual_loss_full_tuned_params.csv'
diag_csv = OUT_DIR / 'dual_loss_full_policy_diagnostics.csv'
report_md = OUT_DIR / 'dual_loss_full_report.md'

runner.write_csv(detailed_csv, detailed_df.to_dict(orient='records'))
runner.write_csv(summary_csv, summary_df.to_dict(orient='records'))
runner.write_csv(tuning_csv, tuning_df.to_dict(orient='records'))
runner.write_csv(diag_csv, diag_df.to_dict(orient='records'))
runner.write_report(report_md, summary_df.to_dict(orient='records'), tuning_df.to_dict(orient='records'), linex_a=LINEX_A)

print('Wrote:', detailed_csv)
print('Wrote:', summary_csv)
print('Wrote:', tuning_csv)
print('Wrote:', diag_csv)
print('Wrote:', report_md)


## Plot: Objective Comparison By Horizon


In [ ]:
for loss_section in ['mse', 'linex']:
    fig, axes = plt.subplots(1, len(HORIZONS), figsize=(5 * len(HORIZONS), 4), sharey=False)
    if len(HORIZONS) == 1:
        axes = [axes]
    for ax, h in zip(axes, HORIZONS):
        sub = summary_df[(summary_df['loss_section'] == loss_section) & (summary_df['horizon'] == float(h))].copy()
        sub = sub.sort_values('objective_mean')
        ax.barh(sub['method'], sub['objective_mean'])
        ax.set_title(f'{loss_section.upper()} | h={h}')
        ax.set_xlabel('Objective Mean')
    plt.tight_layout()
    plt.show()


## Plot: Concentration (Average HHI)


In [ ]:
for loss_section in ['mse', 'linex']:
    fig, axes = plt.subplots(1, len(HORIZONS), figsize=(5 * len(HORIZONS), 4), sharey=True)
    if len(HORIZONS) == 1:
        axes = [axes]
    for ax, h in zip(axes, HORIZONS):
        sub = summary_df[(summary_df['loss_section'] == loss_section) & (summary_df['horizon'] == float(h))].copy()
        sub = sub.sort_values('avg_hhi_mean', na_position='last')
        ax.barh(sub['method'], sub['avg_hhi_mean'])
        ax.set_title(f'{loss_section.upper()} | h={h}')
        ax.set_xlabel('Average HHI')
    plt.tight_layout()
    plt.show()


## RL Diagnostics: Rule Selection Shares


In [ ]:
rule_diag = diag_df[diag_df['method'] == 'RLRuleBandit'].copy()
if not rule_diag.empty:
    shares = (
        rule_diag.groupby(['loss_section', 'horizon', 'action_name'])
        .size()
        .reset_index(name='count')
    )
    shares['share'] = shares.groupby(['loss_section', 'horizon'])['count'].transform(lambda x: x / x.sum())

    for loss_section in ['mse', 'linex']:
        fig, axes = plt.subplots(1, len(HORIZONS), figsize=(5 * len(HORIZONS), 4), sharey=True)
        if len(HORIZONS) == 1:
            axes = [axes]
        for ax, h in zip(axes, HORIZONS):
            sub = shares[(shares['loss_section'] == loss_section) & (shares['horizon'] == float(h))].sort_values('share')
            ax.barh(sub['action_name'], sub['share'])
            ax.set_title(f'Rule Shares | {loss_section.upper()} h={h}')
            ax.set_xlabel('Share')
        plt.tight_layout()
        plt.show()
else:
    print('No rule-bandit diagnostics available.')


## RL Diagnostics: Kappa Trajectory (Example Seed/Horizon)


In [ ]:
kdiag = diag_df[diag_df['method'] == 'RLKappaBandit'].copy()
if not kdiag.empty:
    for loss_section in ['mse', 'linex']:
        seed0 = float(TEST_SEEDS[0])
        h0 = float(HORIZONS[0])
        sub = kdiag[(kdiag['loss_section'] == loss_section) & (kdiag['seed'] == seed0) & (kdiag['horizon'] == h0)]

        fig, ax1 = plt.subplots(figsize=(10, 3))
        ax1.plot(sub['t'], sub['kappa'], label='kappa_t')
        ax1.set_title(f'Kappa Path | {loss_section.upper()} | seed={int(seed0)} h={int(h0)}')
        ax1.set_xlabel('t')
        ax1.set_ylabel('kappa_t')
        ax1.legend(loc='upper left')

        ax2 = ax1.twinx()
        ax2.plot(sub['t'], sub['hhi_t'], color='tab:orange', alpha=0.6, label='HHI_t')
        ax2.set_ylabel('HHI_t')
        ax2.legend(loc='upper right')
        plt.tight_layout()
        plt.show()
else:
    print('No kappa-bandit diagnostics available.')
